In [1]:
from pyomo.environ import ConcreteModel, SolverFactory
                           
from idaes.core import FlowsheetBlock
                           
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
                           
from R32_emimTF2N_VDW import configuration

from idaes.generic_models.unit_models import Flash

from idaes.core.util.model_statistics import degrees_of_freedom

import idaes.logger as idaeslog

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": False,
                           "has_pressure_change": True})

2021-04-27 16:24:22 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [3]:
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 6


In [4]:
m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)
m.fs.F101.inlet.mole_frac_comp[0,'emimTF2N'].fix(0.5)

# m.fs.F101.vap_outlet.pressure.fix(399800)

In [5]:
degrees_of_freedom(m.fs.F101)

1

In [6]:
# Initialize the flash unit
m.fs.F101.initialize()

2021-04-27 16:24:32 [INFO] idaes.init.fs.F101.control_volume.properties_in: Starting initialization
2021-04-27 16:24:34 [INFO] idaes.init.fs.F101.control_volume.properties_in: Dew and bubble point initialization: maxIterations - Maximum Number of Iterations Exceeded..
2021-04-27 16:24:34 [INFO] idaes.init.fs.F101.control_volume.properties_in: Equilibrium temperature initialization completed.
2021-04-27 16:24:36 [INFO] idaes.init.fs.F101.control_volume.properties_in: Phase equilibrium initialization: maxIterations - Maximum Number of Iterations Exceeded..
2021-04-27 16:24:38 [INFO] idaes.init.fs.F101.control_volume.properties_in: Property initialization: maxIterations - Maximum Number of Iterations Exceeded..
2021-04-27 16:24:38 [INFO] idaes.init.fs.F101.control_volume.properties_out: Starting initialization
2021-04-27 16:24:41 [INFO] idaes.init.fs.F101.control_volume.properties_out: Dew and bubble point initialization: maxIterations - Maximum Number of Iterations Exceeded..
2021-04-27 

In [7]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [8]:
status = solver.solve(m, tee = False)

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.


In [9]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value      : Fixed : Bounds
    Pressure Change : 8.9621e+05 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                               Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                    1.0000      0.29429       0.70570  
    mole_frac_comp emimTF2N    0.50000   1.0000e-08       0.70848  
    mole_frac_comp R32         0.50000       1.0000       0.29152  
    temperature                 323.15       305.17        305.17  
    pressure                3.9980e+05   1.2960e+06    1.2960e+06  


In [ ]:
# m.display()

In [ ]:
# m.pprint()